In [2]:
import pandas as pd
import numpy as np

excel_file = "../data/4_GEOMAR-relevant_COG_grantees_and_panel_members_2013-2024.xlsx"

df_excel = pd.read_excel(excel_file, sheet_name="Panel members")

df_excel

,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call
0,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Member,LS,CoG 2014
1,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Chair,LS,CoG 2016
2,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2014
3,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2016
4,Ameli,Nadia,UCL Institute for Sustainable Resources,"Climate Finance. climate finance, networks and...",SH7,Member,SH,CoG 2024
...,...,...,...,...,...,...,...,...
564,Werner,Christiane,University of Freiburg,Ecosystem Physiology. plant ecophysiology and ...,LS8,Member,LS,CoG 2022
565,Zardoya,Rafael,CSIC,molecular systematics; determining the amount ...,LS8,Member,LS,CoG 2014
566,Zardoya,Rafael,CSIC,molecular systematics; determining the amount ...,LS8,Member,LS,CoG 2016
567,Zilitinkevich,Sergej,Finnish Meteorological Institute,boundary layer meteorology; atmospheric physic...,PE10,Member,PE,CoG 2013


In [34]:
def highlight_continuous_members(df):
    """
    Markiere Mitglieder die 4 mal aufeinander im Panel waren.
    Dabei wird eine neue Spalte 'Continuous_Member' hinzugefügt, die True für kontinuierliche Mitglieder und False für andere Mitglieder enthält.
    Hier wird nur das Jahr True markiert, in dem das Mitglied zum 4. Mal in Folge teilgenommen hat.
    Wenn ein Mitglied z.B. 2015, 2016, 2017, 2018 teilgenommen hat, wird nur 2018 als True markiert. Wenn das Mitglied wieder 2020 teilnimmt, wird 2020 nicht markiert, da die Teilnahme nicht kontinuierlich ist.

    Args:
        df (pd.DataFrame): _DataFrame mit Panel-Mitgliedern und deren Teilnahmejahren_
    Returns:
        pd.DataFrame: _DataFrame mit zusätzlicher Spalte 'Continuous_Member'_
    """

    # # add beispiel daten
    # print(df.shape)
    # df = pd.concat([df, pd.DataFrame(
    #     {
    #     "First name": ["Alice", "Alice", "Alice", "Alice", "Alice"],
    #     "Last name": ["Adam", "Adam", "Adam", "Adam", "Adam"],
    #     "Call": ["Panel 2015", "Panel 2016", "Panel 2017", "Panel 2018", "Panel 2019"],
    #     })
    #     ], 
    # ignore_index=True,)
    # print(df.shape)

    
    
    df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True) # Spaltennamen bereinigen
    df["Name"] = df["First name"] + " " + df["Last name"]
    df["Year"] = df["Call"].str.extract(r'(\d{4})').astype(int)
    df = df.sort_values(by=["Name", "Year"])
    df["4x Continuous Member"] = False
    for name, group in df.groupby("Name"):
        years = group["Year"].values
        count = 1
        for i in range(1, len(years)): # Starte bei 1, da wir das vorherige Jahr vergleichen
            if years[i] == years[i-1] + 1: # Überprüfe ob das Jahr auf das vorherige Jahr folgt
                count += 1 # Erhöhe den Zähler für aufeinanderfolgende Jahre
                if count >= 4: # Wenn der Zähler 4 erreicht, markiere das Jahr als kontinuierlich
                    df.loc[(df["Name"] == name) & (df["Year"] == years[i]), "Continuous_Member"] = True
            else:
                count = 1
    df.drop(columns=["Name", "Year"], inplace=True)
    return df

df = highlight_continuous_members(df_excel)
display(df)


,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call,4x Continuous Member
488,Stephanou,Euripides,University of Crete,environmental organic chemistry; Chemistry of ...,PE10,Member,PE,CoG 2014,False
236,Innes,John,University of British Columbia,forestry; forst management; application of the...,LS9,Member,LS,CoG 2014,False
349,Miklosi,Adam,Eötvös Lovrant University Budapest,dog behaviour research; Wolf-dog comparisons; ...,LS8,Member,LS,CoG 2013,False
407,Paytan,Adina,"University of California, Santa Cruz","biogeochemistry, chemical oceanography and pal...",PE10,Member,PE,CoG 2016,False
408,Paytan,Adina,"University of California, Santa Cruz","biogeochemistry, chemical oceanography and pal...",PE10,Member,PE,CoG 2018,False
...,...,...,...,...,...,...,...,...,...
302,López-Urrutia,Ángel,Spanish Institute of Oceanography,marine ecology; marine plankton asas models to...,LS8,Member,LS,CoG 2014,False
303,López-Urrutia,Ángel,Spanish Institute of Oceanography,marine ecology; marine plankton asas models to...,LS8,Member,LS,CoG 2016,False
305,López-Urrutia,Ángel,Spanish Institute of Oceanography,marine ecology; marine plankton asas models to...,LS8,Member,LS,CoG 2018,False
119,López-Urrutia,Ángel,"Spanish Institute of Oceanography, IEO",Marine ecologist. Uses marine plankton as mode...,LS8,Member,LS,CoG 2020,False
